In [1]:
import numpy as np
from scipy.special import expit

In [2]:
def relu(x):
    return np.maximum(x, 0)

def linear(x):
    return x

def neg(x):
    return -1 * x

In [3]:
ACTIVATION_FUNCTIONS = {
    0: relu,
    1: linear,
    2: np.tanh,
    3: expit,
    4: np.abs,
    5: neg
}

In [4]:
class GenNode:
    """
    GenNode stores one node of the tree
    """
    def __init__(self, _input=None, parent=None, children=set(), function=linear, bias=1):
        """
        Initialize:
        - node parent
        - node children
        - activation function
        - default bias (1)
        """
        self._input = _input
        self.parent = parent
        self.children = set(children)
        self.function = function
        self.bias = bias
        
    def add_child(self, child_node):
        self.children.add(child_node)
        
    def add_parent(self, parent_node):
        self.parent = parent_node
        
    def delete_child(self, child_node):
        self.children.remove(child_node)
        
    def change_function(self, function):
        self.function = function   
        
    def calculate(self, _input=None):
        if len(self.children):
            result = np.sum(np.array([child.calculate(_input) for child in self.children]), axis=0)
        else:
            if _input is not None:
                result = _input
            else:
                result = self._input
        return self.function(self.bias + result)
    
    def clone(self, parent):
        node = GenNode(_input=self._input, parent=parent, function=self.function, bias=self.bias)
        return node

### Test node:

In [5]:
node = GenNode()

In [6]:
node.__dict__

{'_input': None,
 'parent': None,
 'children': set(),
 'function': <function __main__.linear(x)>,
 'bias': 1}

In [7]:
cloned = node.clone(None)

In [8]:
cloned.__dict__

{'_input': None,
 'parent': None,
 'children': set(),
 'function': <function __main__.linear(x)>,
 'bias': 1}

In [9]:
node = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[4])
node.calculate()

array([2, 3, 2])

In [10]:
node.calculate()

array([2, 3, 2])

In [11]:
node = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[2])
node.calculate()

array([ 0.96402758,  0.99505475, -0.96402758])

In [12]:
parent = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[2])
child = GenNode(_input=np.array([1,2,-3]), function=ACTIVATION_FUNCTIONS[0])

parent.add_child(child)
child.add_parent(parent)

In [13]:
parent.__dict__

{'_input': array([ 1,  2, -3]),
 'parent': None,
 'children': {<__main__.GenNode at 0x115414110>},
 'function': <ufunc 'tanh'>,
 'bias': 1}

In [14]:
child.__dict__

{'_input': array([ 1,  2, -3]),
 'parent': <__main__.GenNode at 0x115414150>,
 'children': set(),
 'function': <function __main__.relu(x)>,
 'bias': 1}

In [15]:
child.parent.function

<ufunc 'tanh'>

$sum(tanh(1+relu([2,3,-2])))$

In [16]:
parent.calculate()

array([0.99505475, 0.9993293 , 0.76159416])

In [17]:
print(relu([2,3,-2]))
print(np.tanh([3,4,1]))

[2 3 0]
[0.99505475 0.9993293  0.76159416]


---

In [79]:
class GenNet:
    """
    GenNet stores and calculates full net tree
    """
    def __init__(self, start_depth):
        """
        Initialize:
        - all_nodes
        - start_depth
        - root
        """
        self.all_nodes = set()
        self.start_depth = start_depth
        self.root = None
        parent = None
        for i in range(start_depth):
            j = np.random.randint(len(ACTIVATION_FUNCTIONS))
            child = GenNode(parent=parent, function=ACTIVATION_FUNCTIONS[j])
            self.all_nodes.add(child)
            if parent is not None:
                parent.add_child(child)
            else:
                self.root = child
            parent = child

    def all_children(self, node):
        all_children = set()
        for child_node in node.children:
            all_children.update(self.all_children(child_node))

        return all_children
        
    def add_subnet(self, parent_node, child_node, subnet):
        removed_nodes = self.all_children(child_node)
        for node in removed_nodes:
            self.all_nodes.remove(node)
        self.all_nodes.remove(child_node)

        parent_node.add_child(subnet.root)
        for node in subnet.all_nodes:
            self.all_nodes.add(node)
        
    def clone_subnet(self, node, depth=0):
        subnet = GenNet(depth)
        subnet.root = node.clone(parent=None)
        subnet.all_nodes.add(subnet.root)
        for child in node.children:
            cloned_child = child.clone(parent=subnet.root)
            cloned_child.add_parent(subnet.root)
            subnet.root.add_child(cloned_child)
            subnet.all_nodes.update(self._clone_subnet(child, cloned_child))
            
        return subnet
        
    def _clone_subnet(self, root, cloned_root):
        subnet = set([cloned_root])
        for child in root.children:
            cloned_child = child.clone(parent=cloned_root)
            cloned_child.add_parent(cloned_root)
            cloned_root.add_child(cloned_child)
            subnet.update(self._clone_subnet(child, cloned_child))

        return subnet
    
    def change_function(self, node, function):
        node.change_function(function)

    def calculate(self, _input):
        result = self.root.calculate(_input)
        
        return result
    
    def clone(self):
        net = self.clone_subnet(self.root, self.start_depth)

        return net

### Test net

In [48]:
net = GenNet(1)

In [49]:
net.__dict__

{'all_nodes': {<__main__.GenNode at 0x119c1a5d0>},
 'start_depth': 1,
 'root': <__main__.GenNode at 0x119c1a5d0>}

In [50]:
net.root.__dict__

{'_input': None,
 'parent': None,
 'children': set(),
 'function': <function __main__.linear(x)>,
 'bias': 1}

In [51]:
net.calculate(_input=np.array([2,3,0]))

array([3, 4, 1])

In [52]:
net.change_function(net.root, relu)

In [53]:
net.calculate(_input=np.array([1,-2,0]))

array([2, 0, 1])

In [54]:
net = GenNet(2)

In [55]:
for node in net.all_nodes:
    print(node.__dict__)

{'_input': None, 'parent': <__main__.GenNode object at 0x119c2b450>, 'children': set(), 'function': <function neg at 0x1153e1cb0>, 'bias': 1}
{'_input': None, 'parent': None, 'children': {<__main__.GenNode object at 0x119c2b490>}, 'function': <ufunc 'absolute'>, 'bias': 1}


In [56]:
net.calculate(np.array([-2]))

array([2])

In [57]:
net = GenNet(3)

In [58]:
for node in net.all_nodes:
    print(node.__dict__)

{'_input': None, 'parent': <__main__.GenNode object at 0x119c2a990>, 'children': set(), 'function': <function neg at 0x1153e1cb0>, 'bias': 1}
{'_input': None, 'parent': <__main__.GenNode object at 0x119c2a950>, 'children': {<__main__.GenNode object at 0x119c2acd0>}, 'function': <ufunc 'expit'>, 'bias': 1}
{'_input': None, 'parent': None, 'children': {<__main__.GenNode object at 0x119c2a990>}, 'function': <function linear at 0x10e5665f0>, 'bias': 1}


In [59]:
subnet = net.clone_subnet(list(net.all_nodes)[1])

In [60]:
subnet.__dict__

{'all_nodes': {<__main__.GenNode at 0x119c2c690>,
  <__main__.GenNode at 0x119c2c6d0>},
 'start_depth': 0,
 'root': <__main__.GenNode at 0x119c2c690>}

In [61]:
subnet.root.children

{<__main__.GenNode at 0x119c2c6d0>}

---

In [169]:
class GenGenetic:
    def __init__(self, population_size=100, max_depth=10,
                 nets_amount=10, functions=ACTIVATION_FUNCTIONS):
        self.population_size = population_size
        self.population = []
        
        self.functions = functions
        self.max_depth = max_depth
        self.nets_amount = nets_amount
        
    def create_population(self):
        for _ in range(self.population_size):
            j = np.random.randint(1, self.max_depth+1)
            self.population.append(GenNet(j))
        
    def clone(self, i): ### UPDATE ENVIRONMENT
        pass
    
    def cross(self, net_1, net_2):
        net_1 = net_1.clone()
        net_2 = net_2.clone()
        
        i = np.random.randint(len(net_1.all_nodes))
        j = np.random.randint(len(net_2.all_nodes))
        
        node_1 = list(net_1.all_nodes)[i]
        node_2 = list(net_2.all_nodes)[j]
        
        while node_1.parent is None:
            i = np.random.randint(len(net_1.all_nodes))
            node_1 = list(net_1.all_nodes)[i]

        while node_2.parent is None:
            j = np.random.randint(len(net_2.all_nodes))
            node_2 = list(net_2.all_nodes)[j]
        
        subnet_1 = net_1.clone_subnet(node_1)
        subnet_2 = net_2.clone_subnet(node_2)
        
        net_1.add_subnet(node_1.parent, node_1, subnet_2)
        net_2.add_subnet(node_2.parent, node_2, subnet_1)
        
        return [net_1, net_2]
    
    def mutate(self, net_ind):
        net = self.population[net_ind].clone()
        i = np.random.randint(len(net.all_nodes))
        node = list(net.all_nodes)[i]
        function_ind = np.random.randint(len(ACTIVATION_FUNCTIONS))
        new_node = GenNode(_input=node._input, parent=node, 
                           function=ACTIVATION_FUNCTIONS[function_ind], bias=node.bias)
        node.add_child(new_node)
        net.all_nodes.add(new_node)
        
        return net
        
    def step(self, prob=0.25):
        if not isinstance(prob, list):
            prob = [1 - prob * 2] + [prob] * 2
        
        action = np.random.choice(3, p=prob)
        if action == 1:
            i, j = np.random.randint(len(self.population), size=2)

            net_1 = self.population[i]
            while len(net_1.all_nodes) == 1:
                i = np.random.randint(len(self.population))
                net_1 = self.population[i]

            net_2 = self.population[j]
            while len(net_2.all_nodes) == 1:
                j = np.random.randint(len(self.population))
                net_2 = self.population[j]

            self.population = np.concatenate((self.population, self.cross(net_1, net_2)))
        elif action == 2:
            i = np.random.randint(len(self.population))
            self.population.append(self.mutate(i))
            
    def calculate_result(self, _input):
        results = []
        for net in self.population:
            results.append(net.calculate(_input))
        return results
    
    def run(self, _input, num_steps=50, prob=0.25, i=1):
        self.create_population()
        for step in range(num_steps):
            self.step(prob=prob)
            results = self.calculate_result(_input)
            losses = np.array([loss(result) for result in results])
            best_ind = losses.argsort()[-1 * self.nets_amount:][::-1]
            self.population = np.array(self.population)[best_ind].tolist()
            
        return self.population, best_ind

In [170]:
def loss(results, i=1):
    loss = ((results - true_results[i])**2).mean()
    return loss

In [171]:
genetic = GenGenetic(population_size=3)

In [172]:
_input = np.arange(2, 2 + len(true_results[0]))

In [173]:
genetic.run(_input, num_steps=100, prob=0.5)

([<__main__.GenNet at 0x119c4a2d0>,
 array([10,  5,  4,  3,  2,  1,  0,  9,  8,  7]))

## Test

In [174]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [175]:
ort10_coltar10 = pd.read_csv('lin_ort10_coltar10_mean.csv', header=None).values
ort10_colfeat10 = pd.read_csv('lin_ort10_colfeat10_mean.csv', header=None).values
ort20 = pd.read_csv('lin_ort20_mean.csv', header=None).values
ort5_coltar10_colfeat5 = pd.read_csv('lin_ort5_coltar10_colfeat5_mean.csv', header=None).values
ort5_coltar15 = pd.read_csv('lin_ort5_coltar15_mean.csv', header=None).values
ort10_coltar5_colfeat5 = pd.read_csv('lin_ort10_coltar5_colfeat5_mean.csv', header=None).values
ort5_colfeat15 = pd.read_csv('lin_ort5_colfeat15_mean.csv', header=None).values
ort15_colfeat5 = pd.read_csv('lin_ort15_colfeat5_mean.csv', header=None).values
ort5_coltar5_colfeat10 = pd.read_csv('lin_ort5_coltar5_colfeat10_mean.csv', header=None).values

ort10_coltar10_data = pd.read_csv('lin_ort10_coltar10_data.csv', header=None)
ort10_colfeat10_data = pd.read_csv('lin_ort10_colfeat10_data.csv', header=None)
ort20_data = pd.read_csv('lin_ort20_data.csv', header=None)
ort5_coltar10_colfeat5_data = pd.read_csv('lin_ort5_coltar10_colfeat5_data.csv', header=None)
ort5_coltar15_data = pd.read_csv('lin_ort5_coltar15_data.csv', header=None)
ort10_coltar5_colfeat5_data = pd.read_csv('lin_ort10_coltar5_colfeat5_data.csv', header=None)
ort5_colfeat15_data = pd.read_csv('lin_ort5_colfeat15_data.csv', header=None)
ort15_colfeat5_data = pd.read_csv('lin_ort15_colfeat5_data.csv', header=None)
ort5_coltar5_colfeat10_data = pd.read_csv('lin_ort5_coltar5_colfeat10_data.csv', header=None)

In [176]:
true_results = [
    ort10_coltar10.reshape(-1),
    ort10_colfeat10.reshape(-1),
    ort20.reshape(-1),
    ort5_coltar10_colfeat5.reshape(-1),
    ort5_coltar15.reshape(-1),
    ort10_coltar5_colfeat5.reshape(-1),
    ort5_colfeat15.reshape(-1),
    ort15_colfeat5.reshape(-1),
    ort5_coltar5_colfeat10.reshape(-1)
]

In [177]:
genetic = GenGenetic()

In [178]:
_input = np.arange(2, 2 + len(true_results[0]))

In [179]:
genetic.run(_input)

([<__main__.GenNet at 0x1190a5bd0>,
 array([11, 10,  2,  1,  0,  3,  5,  4,  6,  8]))